In [2]:
# import netCDF4
from netCDF4 import Dataset
import numpy as np
import xarray as xr
import pandas as pd
import h5py
import os.path
import datetime, calendar
import scipy.io

In [3]:
from pathlib import Path

In [3]:
### ------------ Opening netCDF Files from WRF-Chem ------------------
# # dataset = Path("C:\Users\chat2\Documents\Spring/ 2019\BEHR_Proj\wrfout_d01_2013-03-01_02-00-00")
# dataset = Path("wrfout_d01_2013-03-01_02-00-00")

In [4]:
# Method 1
# file = Dataset(dataset, 'r')
# file

In [5]:
# Method 2
# xr.open_dataset(dataset)

In [6]:
# ### --------------Opening HDF5 files from BEHR -----------------------

# f = h5py.File('OMI_BEHR-DAILY_US_v3-0B_20050101.hdf', 'r')
# # List all the groups in HDF5 file. Our group is "Data".
# print("Key: ")
# print(list(f.keys()))

# # these are the different swaths
# print()
# a_group_key = list(f.keys())[0]
# data = list(f[a_group_key])
# print("Swaths: ")
# print(data)  

# # this is the data we want to look at
# print()
# print("Data from each swath: ")
# for swath in data:  
#     print(list(f[a_group_key][swath].keys()))

In [7]:
# # Extract date from BEHR filename
# BEHR_filepath = r'C:\Users\chat2\Documents\Spring 2019\BEHR_Proj\OMI_BEHR-DAILY_US_v3-0B_20050101.hdf'
# filename = os.path.basename(BEHR_filepath)
# filedate = filename.split('_')[4]
# print(filedate)
# yr, month, day = int(filedate[0:4]), int(filedate[4:6]), int(filedate[6:8])
# print(yr, month, day)
# print(datetime.datetime(yr, month, day))

In [8]:
# # outdated method, this method dealt with the old BEHR hdf files -------------------
# def BEHR_extract_date(filepath):
#     filename = os.path.basename(filepath)
#     filedate = filename.split('_')[4]
#     yr, month, day = int(filedate[0:4]), int(filedate[4:6]), int(filedate[6:8])
#     return datetime.datetime(yr, month, day)

In [9]:
# BEHR_filepath = r'C:\Users\chat2\Documents\Spring 2019\BEHR_Proj\OMI_BEHR-DAILY_US_v3-0B_20050101.hdf'
# BEHR_extract_date(BEHR_filepath)

In [10]:
def WRF_extract_date(filepath): ## need to edit this function for WRF-Chem files
    filename = os.path.basename(filepath)
    split = filename.split('_')
    filedate = split[2]
    yr, month, day = int(filedate[0:4]), int(filedate[5:7]), int(filedate[8:10])
    hr = int(split[3][0:2])
    return datetime.datetime(yr, month, day, hr)

In [11]:
WRF_filepath = r'wrfout_d01_2013-03-01_02-00-00'
WRF_extract_date(WRF_filepath)

datetime.datetime(2013, 3, 1, 2, 0)

In [12]:
def BEHR_extract_date(filepath):
    filename = os.path.basename(filepath)
    filedate = filename.split('_')[4]
    yr, month, day = int(filedate[0:4]), int(filedate[4:6]), int(filedate[6:8])
    return datetime.datetime(yr, month, day)

In [13]:
BEHR_filepath = r'OMI_BEHR-DAILY_US_v3-0B_20050103.mat'
BEHR_extract_date(BEHR_filepath)

datetime.datetime(2005, 1, 3, 0, 0)

In [4]:
# matfile = h5py.File(r'OMI_BEHR-DAILY_US_v3-0B_20050103.mat','r') #can't open with h5py bc matlab files not saved as v7.3-mat
mat = scipy.io.loadmat('OMI_BEHR-DAILY_US_v3-0B_20050103')
for key,value in mat.items():
    print(key)

__header__
__version__
__globals__
Data
OMI
__function_workspace__


In [5]:
print("Keys: ")
mat_keys = list(mat.keys())
print(mat_keys)

# # print(mat['Data'])
# # print("albedo ocean flag: ")
# # print(mat['Data']['AlbedoOceanFlag'])
# # print("AmfTrop: ")
# # print(mat['Data']['AmfTrop'])
# # print("Solar Zenith Angle: ")
# # print(mat['Data']['SolarZenithAngle'])

# print('Time:')
# print(mat['Data']['Time'])
# print(mat['Data']['Time'].shape)

# # print(list(mat['OMI']))
# # for key,value in mat.items():
# #     print(key)

# # mat['OMI'])

Keys: 
['__header__', '__version__', '__globals__', 'Data', 'OMI', '__function_workspace__']


In [16]:
# # file is "C:\Users\chat2\Documents\Spring 2019\BEHR_Proj\OMI_BEHR-DAILY_US_v3-0B_20050101.hdf"
# hdf_file = Path("OMI_BEHR-DAILY_US_v3-0B_20050101.hdf")
# omi_df = pd.read_hdf(hdf_file, 'Swath2481')
# omi_df

Date: Feb 15, 2019
**Coding work today:**
I set up presudocode to get information from OMI and WRF_Chem pixels (to be coded up). 
Then, I wrote code to assign all corresponding WRF_Chem pixels to each OMI pixel using corresponding location. 
I tested that this works, assuming that I can successfully grab arrays from OMI and WRF_Chem pixel files. 

Note for stored variables involved: 
WRF_Chem pixels have a CENTER variable, which stores long and latitude of the center
OMI pixels have corner variables, which have longs and lats of each corner

(My function "interpolate_WRF_to_OMI(wrf_pixel_arr, omi_pixel_arr)" tests that WRF_Chem pixels' center is in the box formed by OMI pixel corners; this is how the code in "..\BEHR_Proj\BEHR-core-utils\AMF_tools\rProfile_WRF.m" calculates it in the function "avg_apriori()".)

Essentially, we want to create a map of all the WRF-Chem pixels' measurements onto a grid of OMI tiles. 
THis map can be created visually later, after I create the dataframes. 


**TBD:**
Need to align all the WRF_Chem_pixels with matching DATE_TIME as well, with the corresponding OMI tiles.
Another factor to account for: WRF-Chem pixels have fixed location, while OMI locations are constantly changing. 

Notes about variables: 
WRF-Chem pixels have fixed location, while OMI locations are constantly changing. 
WRF-Chem pixel date can be found in filename, calc hour using "TIME" variable

Create a dataframe with OMI pixel as the primary index. 
First columns will hold OMI_pixel_index, OMI_pixel_time (might need to use indices for time to refer to another table with diff times, avoids many duplicates)
Secondary columns will hold list of WRF_Chem_pixel_indices.

Create a second dataframe with WRF_Chem_pixel_indices as primary index.
Columns will hold the different measurements we want to gather for each WRF_Chem_pixel (air pressure, temp, and RH)

In the future, we can create a map of all the WRF-Chem pixels' measurements essentially superimposed onto a grid of OMI tiles. 

**(TLDR) Needs to be done next time:**
Align dates, (in addition to location), for WRF_Chem pixels and OMI pixels. 
Create dataframes for easy access to information relating WRF_Chem pixels and OMI tiles. 

In [ ]:
def open_files(filename):
    #open files
    dataset = Path(filename)
    file = Dataset(dataset, 'r')
    return file
    
#Quick data structure questions:
# What's the best way to store the pixel corner data - 
# FOr java, I would store linked list of corners for each pixel
# python equivalent is probably using direct variable attribute or an ordered array for each pixel

# to quickly calc which WRF_Chem pixels belong to which OMI pixels, can create numbering scheme for pixels (gridding)
# alternatively, could use naming scheme of pixel files -> but have time and location in LONG file name

    
# trying to match closest location and date_time among OMI and WRF_Chem pixels
# use WRF_Chem pixels' corners that are in the OMI grid box
def extract_info_OMI(omi_pixel):
    # getting all info from OMI pixel into an array
    # input: OMI pixel file as a Dataset object (sort of like a Python dictionary)
    
    # output: array of corners arrays of longitudes and latitudes and date: [lons, lats, date]
    # format of lons and lats arrays: [upper_left, upper_right, lower_left, lower_right]
    return []

    
    
def extract_info_WRF_Chem(wrf_pixel):
    # getting all info from WRF-Chem pixel into an array
    # input: wrf_chem pixel file
    # output: array of corners and date [center_lon, center_lat, date]
#     log = dict()
#     log['lon'] = omi_pixel[]
#     log['lat'] = omi_pixel['CEN_LAT']
#     log['date'] = omi_pixel[]
#     return log
    
    
    return [omi_pixel['CEN_LON'], omi_pixel['CEN_LAT'], omi_pixel[]]
#     return []

In [ ]:
def interpolate_WRF_to_OMI(wrf_pixel_arr, omi_pixel_arr): # or take A and O as input
    # returns true if wrf_pixel belongs to this OMI pixel, OR can set OMI pixel attribute to true
    # checks that wrf_pixel's center lies within the box formed by the OMI pixel's corners
    
    # lon and lat of wrf pixel's center
    wrf_lon, wrf_lat = wrf_pixel_arr[0], wrf_pixel_arr[1]
    
    # longitude and latitude arrays of OMI pixels
    omi_lons, omi_lats = omi_pixel_arr[0], omi_pixel_arr[1]
    min_lon, max_lon = min(omi_lons), max(omi_lons)
    min_lat, max_lat = min(omi_lats), max(omi_lats)
    
    return (wrf_lon > min_lon and wrf_lon < max_lon and 
            wrf_lat > min_lat and wrf_lon < max_lat)

In [ ]:
# Tests for func interpolate_WRF_to_OMI(wrf_pixel_arr, omi_pixel_arr)
A = [0, 0, 12]
O = [[-1, 1, 1, -1], [1, 1, -1, -1], 12]
interpolate_WRF_to_OMI(A, O) #should return True

A = [0, 0, 12]
O = [[-1, 1, 2, 0], [1, 1, -1, -1], 12]
interpolate_WRF_to_OMI(A, O) #should return True

A = [0, 0, 12]
O = [[-1, 1, 1, -1], [1, 1, 0, 0], 12]
interpolate_WRF_to_OMI(A, O) #should return False

In [ ]:
## Pseudocode for how code might interact... (somewhat wrong now given new information about matching locations of pixels)
# assume A is a wrf pixel, O is a OMI pixel
    
# OMI = []
# for all OMI pixels:
    # extract_info_OMI()
    
A_info_arr = extract_info_WRF_Chem(A)
O_info_arr = extract_info_OMI(O) #done for all the OMI Pixels
# for all nearby OMI pixels, o:   #determine "nearby" location by center of OMI Pixel
    if interpolate_WRF_to_OMI(A, O):
        # wrf pixel belongs to this OMI pixel's measurements

    